In [ ]:
# Plot horizontal planes and vertical columns within a specified patch of latitude and longitude.

# input file locations:
wd = '/lustre/scratch2/turquoise/mpeterse/runs'
dirName1 = 'c70x'
dirName2 = 'c70y'
fileName = 'output/output.0001-01-01_00.00.00.nc'
meshFileName = 'init.nc'

iTime = 5
kLevel = 20

# output file location, for plots:
writeOutput = False
outputDir='figures'

# Variables: single lines
varList = ['salinity', 'vertTransportVelocityTop','divergence']

# title and labels
titleText = ''
xLabel = 'time, days runs:'+dirName1+','+dirName2
legend1="ocean/develop"
legend2="fully split FCT"
endTime = 360

# line types:
lt = '-:'

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numpy as np
import xarray as xr
import subprocess
import os 
# custom libraries
from simple_plotting import plot_horiz, plot_var, rad2deg, deg2rad, plot_poly
# define plot dimensions
%matplotlib inline  
plt.rcParams["figure.figsize"] = (12.,9.)

In [ ]:
# open the first netCDF file for reading.
filePathName = wd + '/' + dirName1 + '/' + fileName
print('Reading: ' + filePathName)
ds = xr.open_dataset(filePathName)

meshPathName = wd + '/' + dirName1 + '/' + meshFileName
print('Reading: ' + meshPathName)
mesh = xr.open_dataset(meshPathName).squeeze()

In [ ]:
# join mesh and sim
dsvars = ds.variables.keys()
meshvars = mesh.variables.keys()
dropvars = np.intersect1d(dsvars, meshvars)
ds = ds.merge(mesh.drop(dropvars), join='left')
ds = ds.set_coords(['lonCell','latCell'])
zcoord = ds.refZMid.values

In [ ]:
def plot_profile(variable, atime=-1, maxLayers=60, dc=None):
    for kk in np.arange(np.minimum(maxLayers,variable.shape[-1])):
        plt.figure()
        vartime = variable[atime,:,kk]
        vartime[np.where(vartime < -1e33)] = np.nan
        pc = plot_poly(vartime, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), cmap='seismic', colorbar=False)
        plt.title(variable.name + ' Layer %d (%f m)'%(kk, ds.refZMid[kk]))
        if dc is not None:
            #pc.set_clim(np.nanmean(vartime)-dc, np.nanmean(vartime)+dc)
            pc.set_clim(-dc, +dc)
            cb = plt.colorbar(pc)

In [ ]:
def plot_var(variable, time=-1, maxdepth=-5300.):
    if not 'nCells' in variable.dims:
        return
    variable = variable.isel(nCells=np.arange(10))
    nmax = np.minimum(ds.maxLevelCell.values.max(), np.where(zcoord < maxdepth)[0][0])
    for ii in np.arange(len(variable.nCells)):
        lon = rad2deg(variable.lonCell.values[ii])
        lat = rad2deg(variable.latCell.values[ii])
        plt.plot(variable[time,ii,:nmax],zcoord[:nmax], '-', label='lon=%.4f, lat=%.4f'%(lon,lat))
    plt.ylabel('Depth (m)')
    plt.legend(loc='best',ncol=2)
    attrs = variable.attrs
    plt.xlabel(attrs['long_name'][0:100] + ' ' + attrs['units'])
    

In [ ]:
# define plotting location and number of cells
plotlon = 128
plotlat = 11.5
ntotal = 100

dist = (ds.lonCell.values - deg2rad(plotlon))**2.0 + (ds.latCell.values - deg2rad(plotlat))**2.0
points = np.where(dist < np.sort(dist)[ntotal])[0] # np.min(dist))[0]
dspoints = ds.isel(nCells=points)

plt.plot(rad2deg(ds.lonCell), rad2deg(ds.latCell),'.',ms=0.1)
plt.plot(rad2deg(dspoints.lonCell), rad2deg(dspoints.latCell), 'r.')

In [ ]:
var = dspoints.maxLevelCell[:]
pc = plot_poly(var, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
#pc.set_clim((var.mean()-2e-5,var.mean()+2e-5))
plt.colorbar(pc)

In [ ]:
salinity = dspoints.salinity[iTime,:,kLevel]
pc = plot_poly(salinity, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
pc.set_clim((salinity.mean()-.1,salinity.mean()+.1))
plt.colorbar(pc)

In [ ]:
var = dspoints.vertTransportVelocityTop[iTime,:,kLevel+1]
pc = plot_poly(var, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
pc.set_clim((var.mean()-2e-5,var.mean()+2e-5))
plt.colorbar(pc)

In [ ]:
var = dspoints.vertVelocityTop[iTime,:,kLevel+1]
pc = plot_poly(var, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
pc.set_clim((var.mean()-4e-5,var.mean()+4e-5))
plt.colorbar(pc)

In [ ]:
divergence = dspoints.divergence[iTime,:,kLevel]
pc = plot_poly(divergence, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
pc.set_clim((-1e-6,1e-6))
plt.colorbar(pc)

In [ ]:
plot_var(dspoints.salinity,time=iTime)

In [ ]:
plot_var(dspoints.vertTransportVelocityTop)

In [ ]:
plot_var(dspoints.divergence)

In [ ]:
# Horizontal sections at every level:
#plot_profile(dspoints.salinity, atime=iTime, dc=0.2)

In [ ]:
# Horizontal sections at every level:
#plot_profile(dspoints.vertTransportVelocityTop, atime=iTime, dc=1e-4)

In [ ]:
# define plotting location and number of cells
plotlon = 171
plotlat = 0
ntotal = 100

dist = (ds.lonCell.values - deg2rad(plotlon))**2.0 + (ds.latCell.values - deg2rad(plotlat))**2.0
points = np.where(dist < np.sort(dist)[ntotal])[0] # np.min(dist))[0]
dspoints = ds.isel(nCells=points)

plt.plot(rad2deg(ds.lonCell), rad2deg(ds.latCell),'.',ms=0.1)
plt.plot(rad2deg(dspoints.lonCell), rad2deg(dspoints.latCell), 'r.')

In [ ]:
var = dspoints.maxLevelCell[:]
pc = plot_poly(var, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
#pc.set_clim((var.mean()-2e-5,var.mean()+2e-5))
plt.colorbar(pc)

In [ ]:
salinity = dspoints.salinity[iTime,:,kLevel]
pc = plot_poly(salinity, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
pc.set_clim((salinity.mean()-.2,salinity.mean()+.2))
plt.colorbar(pc)

In [ ]:
divergence = dspoints.divergence[iTime,:,kLevel]
pc = plot_poly(divergence, dspoints.verticesOnCell.values-1, dspoints.nEdgesOnCell.values, 
               rad2deg(ds.lonVertex.values), rad2deg(ds.latVertex.values), colorbar=False, cmap='seismic')
pc.set_clim((-1e-6,1e-6))
plt.colorbar(pc)

In [ ]:
plot_var(dspoints.salinity,time=iTime)

In [ ]:
plot_var(dspoints.vertVelocityTop)

In [ ]:
plot_var(dspoints.vertTransportVelocityTop)

In [ ]:
plot_var(dspoints.divergence)

In [ ]:
# Horizontal sections at every level:
#plot_profile(dspoints.salinity, atime=iTime, dc=2e-4)

In [ ]:
# Horizontal sections at every level:
#plot_profile(dspoints.vertTransportVelocityTop, atime=iTime, dc=2e-4)